In [ ]:
from sqlalchemy import create_engine


engine = create_engine('mysql+pyodbc://telecom')
%load_ext sql
%sql engine
%config SqlMagic.autopandas = True

In [ ]:
%sql SHOW tables

# Radius and Eccentricity
```sql
CREATE TABLE sample_center_201308 (
	SELECT 
        SERV_ID, ACC_NBR, 
        AVG(X) AS CENTER_X, 
        AVG(Y) as CENTER_Y
    FROM sample_sight_201308
    GROUP BY SERV_ID, ACC_NBR
);
CREATE INDEX individual_center_201308 ON sample_center_201308(SERV_ID, ACC_NBR);

CREATE TABLE sample_deviate_201308 (
	SELECT 
        A.*, B.CENTER_X, B.CENTER_Y, 
        A.X-B.CENTER_X AS X_DEVIATE, 
        A.Y-B.CENTER_Y AS Y_DEVIATE
	FROM sample_sight_201308 A LEFT JOIN sample_center_201308 B
	ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.ACC_NBR
);
CREATE INDEX individual_deviate_201308 ON sample_deviate_201308(SERV_ID, ACC_NBR);
DROP TABLE sample_center_201308;

CREATE TABLE var_rog_201308 (
	SELECT 
        SERV_ID, ACC_NBR, 
        AVG(SQRT(POWER(X_DEVIATE,2)+POWER(Y_DEVIATE,2))) AS ROG, 
		SUM(POWER(X_DEVIATE,2))/(COUNT(1)-1) AS S2_XX, 
        SUM(POWER(Y_DEVIATE,2))/(COUNT(1)-1) AS S2_YY, 
        SUM(X_DEVIATE*Y_DEVIATE)/(COUNT(1)-1) AS S2_XY
	FROM sample_deviate_201308
	GROUP BY SERV_ID, ACC_NBR
);

CREATE TABLE var_eccen_201308 (
	SELECT SERV_ID, ACC_NBR, ROG, 
		SQRT(1-(S2_XX+S2_YY-SQRT(POWER(S2_XX-S2_YY,2)+4*POWER(S2_XY,2)))/(S2_XX+S2_YY+SQRT(POWER(S2_XX-S2_YY,2)+4*POWER(S2_XY,2)))) AS ECCEN
	FROM var_rog_201308
);
CREATE INDEX individual_eccen_201308 ON var_eccen_201308(SERV_ID, ACC_NBR);
DROP TABLE var_rog_201308;
```

# Entropy and Center-Frequency
```sql
CREATE TABLE sight_ntotal_201308 (
	SELECT 
        SERV_ID, ACC_NBR, 
        COUNT(1) AS n
	FROM sample_sight_201308
	GROUP BY SERV_ID, ACC_NBR
);
CREATE INDEX individual_ntotal_201308 ON sight_ntotal_201308(SERV_ID, ACC_NBR);

CREATE INDEX xy_samplesight_201308 ON sample_sight_201308(X, Y);
CREATE TABLE sight_ktime_201308 (
	SELECT 
        SERV_ID, ACC_NBR, X, Y, 
        COUNT(1) AS k
	FROM sample_sight_201308
	GROUP BY SERV_ID, ACC_NBR, X, Y
);
CREATE INDEX individual_ktime_201308 ON sight_ktime_201308(SERV_ID, ACC_NBR);

CREATE TABLE sample_prob_201308 (
	SELECT 
        A.*, B.n, 
        CONVERT(A.k,DECIMAL)/CONVERT(B.n,DECIMAL) AS p
	FROM sight_ktime_201308 A LEFT JOIN sight_ntotal_201308 B
	ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.ACC_NBR
);
CREATE INDEX individual_prob_201308 ON sample_prob_201308(SERV_ID, ACC_NBR);
DROP TABLE sight_ktime_201308;

CREATE TABLE var_ent_201308 (
	SELECT SERV_ID, ACC_NBR, ABS(SUM(p*(LOG(p)/LOG(2)))) AS ENT
	FROM sample_prob_201308
	GROUP BY SERV_ID, ACC_NBR
);
CREATE INDEX individual_ent_201308 ON var_ent_201308(SERV_ID, ACC_NBR);

CREATE INDEX center_samplesight_201308 ON sample_sight_201308(CENTER_FLAG);
CREATE TABLE sight_center_201308 (
	SELECT 
        SERV_ID, ACC_NBR, 
        COUNT(1) AS n_CENTER
	FROM sample_sight_201308
	WHERE CENTER_FLAG=1
	GROUP BY SERV_ID, ACC_NBR
);
CREATE INDEX individual_center_201308 ON sight_center_201308(SERV_ID, ACC_NBR);

CREATE TABLE var_freq_center_201308 (
	SELECT 
        A.*, B.n_CENTER, 
        CONVERT(IFNULL(B.n_CENTER,0),DECIMAL)/CONVERT(A.n,DECIMAL)*100 AS FREQ_CENTER
	FROM sight_ntotal_201308 A LEFT JOIN sight_center_201308 B
	ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.ACC_NBR
);
CREATE INDEX individual_freqcenter_201308 ON var_freq_center_201308(SERV_ID, ACC_NBR);
DROP TABLE sight_ntotal_201308;
DROP TABLE sight_center_201308;
```

# Output
```sql
CREATE TABLE var_201308 (
	SELECT  
		#(CONVERT(SUBSTR(B.MONTH_NO,1,4),DECIMAL)-CONVERT(SUBSTR(B.CERT_NBR,10,4),DECIMAL)-1)+(12-CONVERT(SUBSTR(B.CERT_NBR,15,2),DECIMAL)+CONVERT(SUBSTR(B.MONTH_NO,5,2),DECIMAL))/12 AS AGE, 
		#SUBSTR(B.CERT_NBR,21,1) AS GENDER_MALE, 
        A.FREQ_CENTER,
        CASE WHEN A.FREQ_CENTER=0 THEN 0 ELSE 1 END AS CENTER, 
		B.SERV_ID, B.MSISDN, B.MONTH_NO, B.CERT_NBR, B.EMPLOY_FLAG, B.STUDENT_FLAG, B.RURAL_FLAG, B.ARPU, 
        B.TERMINAL_MODEL AS TERMINAL, 
        B.TERMINAL_BRAND AS BRAND, 
        B.TERMINAL_SMART_FLAG AS SMART, 
        B.TERMINAL_LEVEL AS LEVEL, 
        B.TERMINAL_PRICE AS PRICE, 
		B.PD_EVDO_FLAG AS EVDO,
        B.CONTRACT_FLAG AS CONTRACT,
		C.ROG, C.ECCEN, 
        D.ENT,
	FROM var_freq_center_201308 A
	LEFT JOIN sample_user_201308 B ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.MSISDN
	LEFT JOIN var_eccen_201308 C ON A.SERV_ID=C.SERV_ID AND A.ACC_NBR=C.ACC_NBR
	LEFT JOIN var_ent_201308 D ON A.SERV_ID=D.SERV_ID AND A.ACC_NBR=D.ACC_NBR
);
DROP TABLE var_eccen_201308;
DROP TABLE var_ent_201308;
CREATE INDEX individual_var_201308 ON var_201308(SERV_ID, MSISDN);
```

# network
```sql
CREATE INDEX callinged_201308 ON serv_acct_item_0838_201308(CALLING_NBR, CALLED_NBR);
CREATE INDEX etl_201308 ON serv_acct_item_0838_201308(ETL_TYPE_ID);
CREATE TABLE presample_link_201308 (
	SELECT DISTINCT A.CALLING_NBR, A.CALLED_NBR
	FROM serv_acct_item_0838_201308 A JOIN var_201308 B
	ON A.SERV_ID=B.SERV_ID AND A.ACC_NBR=B.MSISDN
	WHERE 
        A.CALLING_NBR!=A.CALLED_NBR AND
		(A.CALLING_NBR NOT LIKE 'NotMobile_%' AND A.CALLED_NBR NOT LIKE 'NotMobile_%') AND
		(A.ETL_TYPE_ID='21' OR A.ETL_TYPE_ID='31' OR A.ETL_TYPE_ID='24' OR A.ETL_TYPE_ID='34') AND
		(A.LAC!='-1' AND A.LAC!='0') AND (A.CELL_ID!='-1' AND A.CELL_ID!='0')
);
CREATE INDEX callinged_prelink_201308 ON presample_link_201308(CALLING_NBR, CALLED_NBR);

CREATE TABLE sample_prelink_201308 (
	SELECT
		IF(CALLING_NBR<CALLED_NBR,CALLING_NBR,CALLED_NBR) AS NBR_1, 
		IF(CALLING_NBR<CALLED_NBR,CALLED_NBR,CALLING_NBR) AS NBR_2
	FROM presample_link_201308
);
CREATE INDEX nbr1_prelink_201308 ON sample_prelink_201308(NBR_1);
CREATE INDEX nbr2_prelink_201308 ON sample_prelink_201308(NBR_2);
CREATE INDEX nbr12_prelink_201308 ON sample_prelink_201308(NBR_1, NBR_2);
DROP TABLE presample_link_201308;

#ALTER TABLE var_201308 ADD PRIMARY KEY(MSISDN);
CREATE INDEX nbr_var_201308 ON var_201308(MSISDN);
CREATE TABLE link_201308 (
	SELECT DISTINCT * FROM sample_prelink_201308
	WHERE
		NBR_1 IN (
			SELECT MSISDN FROM var_201308
		)
		AND
		NBR_2 IN (
			SELECT MSISDN FROM var_201308
		)
	ORDER BY NBR_1, NBR_2
);
DROP TABLE sample_prelink_201308;

```